In [1]:
import pandas as pd
import xlsxwriter
from datetime import date


In [2]:
df_sp_targeting = pd.read_excel('../sample_files/Sponsored Products Targeting report .xlsx',index_col=False)
df_sp_targeting.head(5)

C:\Users\dvkhai2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Start Date,End Date,Portfolio name,Currency,Campaign Name,Ad Group Name,Targeting,Match Type,Impressions,Clicks,...,Total Advertising Cost of Sales (ACOS),Total Return on Advertising Spend (ROAS),7 Day Total Sales,7 Day Total Orders (#),7 Day Total Units (#),7 Day Conversion Rate,7 Day Advertised SKU Units (#),7 Day Other SKU Units (#),7 Day Advertised SKU Sales,7 Day Other SKU Sales
0,2022-10-05,2022-12-10,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,hanging woven wall basket,EXACT,12019,26,...,0.155922,6.413447,165.98,2,2,0.076923,2,0,165.98,0.00
1,2022-10-04,2022-11-13,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,wall basket decor,EXACT,41544,110,...,1.533556,0.652079,47.98,1,2,0.009091,0,2,0.00,47.98
2,2022-10-08,2022-12-05,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,boho wall baskets,EXACT,16170,72,...,0.207076,4.829144,464.95,5,5,0.069444,4,1,359.96,104.99
3,2022-10-04,2022-12-10,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,black wall basket,EXACT,5614,13,...,0.051812,19.300530,254.96,4,4,0.307692,4,0,254.96,0.00
4,2022-10-05,2022-11-20,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,large wall basket decor boho,EXACT,18,0,...,NaN,NaN,0.00,0,0,NaN,0,0,0.00,0.00


In [59]:
def read_bulk_report_CSP(excel_file=''):
    bulk_df = pd.read_excel(excel_file,index_col=False)
    df_process = bulk_df[['Campaign Id','Ad Group Id','Portfolio Id','Keyword Id (Read only)','Product Targeting Id (Read only)','Campaign Name (Informational only)','Ad Group Name (Informational only)','Portfolio Name (Informational only)','Keyword Text','Campaign State (Informational only)','Ad Group State (Informational only)','Bid']]
    df_process.columns = ['Campaign Id','Ad Group Id','Portfolio Id','Keyword Id','Product Targeting Id','Campaign Name','Ad Group Name','Portfolio Name','Keyword Text','Campaign State','Ad Group State','Bid']
    df_process = df_process[df_process['Campaign State']!='archived'][df_process['Ad Group State']!='archived']
    # df_process = df_process[df_process['Ad Group State']!='archived']
    return df_process
    
def get_campid_toDF(filtered_df:pd.DataFrame,bulk_df:pd.DataFrame):

    bulk_df_campaign = get_campaign_infor(bulk_df)
    bulk_df_adgroup = get_adgroup_infor(bulk_df)

    filtered_df['Campaign Name'] = filtered_df['Campaign Name'].astype("string")
    filtered_df['Ad Group Name'] = filtered_df['Ad Group Name'].astype("string")
    bulk_df_campaign['Campaign Name'] = bulk_df_campaign['Campaign Name'].astype("string")
    bulk_df_campaign['Ad Group Name'] = bulk_df_campaign['Ad Group Name'].astype("string")
    df_ver1 = pd.merge(filtered_df, bulk_df_campaign, how='left',left_on = ['Campaign Name','Ad Group Name'],right_on = ['Campaign Name','Ad Group Name'])
    # df_ver2 = pd.merge(df_ver1, bulk_df_adgroup, how='left',left_on = 'Ad Group Name',right_on = 'Ad Group Name')
    print('Length original: ',len(filtered_df))
    print('Length camp merge: ',len(df_ver1))
    # print('Lengh adgroup merge: ',len(df_ver2))
    return df_ver1

def get_keyword_Bid(filter_withCamID:pd.DataFrame,bulk_df:pd.DataFrame):

    filter_withCamID['Campaign Name'] = filter_withCamID['Campaign Name'].astype("string")
    filter_withCamID['Ad Group Name'] = filter_withCamID['Ad Group Name'].astype("string")
    filter_withCamID['Targeting'] = filter_withCamID['Targeting'].astype("string")
    bulk_df['Campaign Name'] = bulk_df['Campaign Name'].astype("string")
    bulk_df['Ad Group Name'] = bulk_df['Ad Group Name'].astype("string")
    bulk_df['Keyword Id'] = bulk_df['Keyword Id'].astype("string")
    df_ver1 = pd.merge(filter_withCamID, bulk_df, how='left',left_on = ['Campaign Name','Ad Group Name','Targeting'],right_on = ['Campaign Name','Ad Group Name','Keyword Text'])
    print("Len Final Merge: ",df_ver1)
    return df_ver1


# 'Keyword Id','Product Targeting Id','Bid'
def get_campaign_infor(df_bulk:pd.DataFrame):
    df_campaign_info = df_bulk[['Campaign Id','Campaign Name','Ad Group Id','Ad Group Name']]
    df_campaign_info.drop_duplicates(inplace=True)
    df_campaign_info.reset_index(inplace=True)
    return df_campaign_info

def get_adgroup_infor(df_bulk:pd.DataFrame):
    df_adgroup_info = df_bulk[['Ad Group Id','Ad Group Name']]
    df_adgroup_info.drop_duplicates(inplace=True)
    df_adgroup_info.reset_index(inplace=True)
    return df_adgroup_info

def filter_SP_downbid(CSP_df: pd.DataFrame,acos,clicks,spend):
    filerted_df_1 = CSP_df[CSP_df['Total Advertising Cost of Sales (ACOS) ']>acos]
    filtered_df_2 = CSP_df[CSP_df['Clicks']>clicks][CSP_df['7 Day Total Sales ']==0]
    filtered_df_3 = CSP_df[CSP_df['Spend']>spend][CSP_df['7 Day Total Sales ']==0]
    list_frames = [filerted_df_1,filtered_df_2,filtered_df_3]
    filtered_df = pd.concat(list_frames)
    filtered_df.drop_duplicates(inplace=True)
    return filtered_df


In [30]:
df_bulk = read_bulk_report_CSP(excel_file='../sample_files/BULK Sponsored Products Campaigns.xlsx')

C:\Users\dvkhai2\AppData\Local\Temp\ipykernel_14748\3171583516.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_process = df_process[df_process['Campaign State']!='archived'][df_process['Ad Group State']!='archived']


In [38]:
df_bulk.columns

Index(['Campaign Id', 'Ad Group Id', 'Portfolio Id', 'Keyword Id',
       'Product Targeting Id', 'Campaign Name', 'Ad Group Name',
       'Portfolio Name', 'Keyword Text', 'Campaign State', 'Ad Group State',
       'Bid'],
      dtype='object')

In [32]:
df_bulk.head(5)

,Campaign Id,Ad Group Id,Portfolio Id,Keyword Id,Product Targeting Id,Campaign Name,Ad Group Name,Portfolio Name,Keyword Text,Campaign State,Ad Group State,Bid
0,268276414038554,NaN,2.789090e+14,NaN,NaN,CC-0011-0001 US SP AUTO Chicnchill 20220919 HUY 1,NaN,2. DCR1,NaN,paused,NaN,NaN
1,268276414038554,NaN,NaN,NaN,NaN,CC-0011-0001 US SP AUTO Chicnchill 20220919 HUY 1,NaN,2. DCR1,NaN,paused,NaN,NaN
2,268276414038554,NaN,NaN,NaN,NaN,CC-0011-0001 US SP AUTO Chicnchill 20220919 HUY 1,NaN,2. DCR1,NaN,paused,NaN,NaN
3,268276414038554,8.208207e+13,NaN,NaN,NaN,CC-0011-0001 US SP AUTO Chicnchill 20220919 HUY 1,CC-0011-0001 US SP AUTO Chicnchill 20221003 HUY 1,2. DCR1,NaN,paused,enabled,NaN
4,268276414038554,8.208207e+13,NaN,NaN,NaN,CC-0011-0001 US SP AUTO Chicnchill 20220919 HUY 1,CC-0011-0001 US SP AUTO Chicnchill 20221003 HUY 1,2. DCR1,NaN,paused,enabled,NaN


In [39]:
len(df_bulk)

48634

In [40]:
df_filter_downbid = filter_SP_downbid(CSP_df=df_sp_targeting,acos=0.25,clicks=8,spend=10)
df_filter_downbid.head(5)

C:\Users\dvkhai2\AppData\Local\Temp\ipykernel_14748\2266666949.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df_2 = CSP_df[CSP_df['Clicks']>clicks][CSP_df['7 Day Total Sales ']==0]
C:\Users\dvkhai2\AppData\Local\Temp\ipykernel_14748\2266666949.py:39: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df_3 = CSP_df[CSP_df['Spend']>spend][CSP_df['7 Day Total Sales ']==0]


,Start Date,End Date,Portfolio name,Currency,Campaign Name,Ad Group Name,Targeting,Match Type,Impressions,Clicks,...,Total Advertising Cost of Sales (ACOS),Total Return on Advertising Spend (ROAS),7 Day Total Sales,7 Day Total Orders (#),7 Day Total Units (#),7 Day Conversion Rate,7 Day Advertised SKU Units (#),7 Day Other SKU Units (#),7 Day Advertised SKU Sales,7 Day Other SKU Sales
1,2022-10-04,2022-11-13,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,wall basket decor,EXACT,41544,110,...,1.533556,0.652079,47.98,1,2,0.009091,0,2,0.00,47.98
5,2022-10-04,2022-11-11,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,wall hanging baskets,EXACT,7014,21,...,0.695342,1.438142,34.99,1,1,0.047619,0,1,0.00,34.99
21,2022-10-27,2022-11-06,3. DCR2,USD,CC-0027-0003 US SP ASIN Chicnchill 20221028 TO...,CC-0027-0003 SP US ASIN Chicnchill 20221028 TO...,"asin=""B084VL1XFN""",-,5519,37,...,0.873291,1.145094,59.98,2,2,0.054054,2,0,59.98,0.00
28,2022-10-05,2022-12-13,3. DCR2,USD,CC-0027-0003 US SP ASIN Chicnchill 20221028 TO...,CC-0027-0003 SP US ASIN Chicnchill 20221028 TO...,"asin=""B09771BDVV""",-,46578,304,...,0.439239,2.276666,532.33,16,17,0.052632,9,8,264.91,267.42
41,2022-10-05,2022-10-27,3. DCR2,USD,CC-0027-0003 US SP ASIN Chicnchill 20221028 TO...,CC-0027-0003 SP US ASIN Chicnchill 20221028 TO...,"asin=""B092ZMT3KQ""",-,8454,52,...,0.785165,1.273618,64.98,2,2,0.038462,2,0,64.98,0.00


In [41]:
len(df_filter_downbid)

633

In [ ]:
### Merge 2 Data Frames
df_merge = get_campid_toDF(filtered_df=df_filter_downbid,bulk_df=df_bulk)
df_merge

In [49]:
df_merge

,Start Date,End Date,Portfolio name,Currency,Campaign Name,Ad Group Name,Targeting,Match Type,Impressions,Clicks,...,7 Day Total Orders (#),7 Day Total Units (#),7 Day Conversion Rate,7 Day Advertised SKU Units (#),7 Day Other SKU Units (#),7 Day Advertised SKU Sales,7 Day Other SKU Sales,index,Campaign Id,Ad Group Id
0,2022-10-04,2022-11-13,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,wall basket decor,EXACT,41544,110,...,1,2,0.009091,0,2,0.00,47.98,11758.0,2.722893e+14,3.844219e+13
1,2022-10-04,2022-11-11,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,wall hanging baskets,EXACT,7014,21,...,1,1,0.047619,0,1,0.00,34.99,11758.0,2.722893e+14,3.844219e+13
2,2022-10-27,2022-11-06,3. DCR2,USD,CC-0027-0003 US SP ASIN Chicnchill 20221028 TO...,CC-0027-0003 SP US ASIN Chicnchill 20221028 TO...,"asin=""B084VL1XFN""",-,5519,37,...,2,2,0.054054,2,0,59.98,0.00,5505.0,1.423994e+14,2.442783e+14
3,2022-10-05,2022-12-13,3. DCR2,USD,CC-0027-0003 US SP ASIN Chicnchill 20221028 TO...,CC-0027-0003 SP US ASIN Chicnchill 20221028 TO...,"asin=""B09771BDVV""",-,46578,304,...,16,17,0.052632,9,8,264.91,267.42,5505.0,1.423994e+14,2.442783e+14
4,2022-10-05,2022-10-27,3. DCR2,USD,CC-0027-0003 US SP ASIN Chicnchill 20221028 TO...,CC-0027-0003 SP US ASIN Chicnchill 20221028 TO...,"asin=""B092ZMT3KQ""",-,8454,52,...,2,2,0.038462,2,0,64.98,0.00,5505.0,1.423994e+14,2.442783e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,2022-10-07,2022-10-29,2. DCR1,USD,CC-0004-0012 US SP KCST Chicnchill 20221007 HUY 1,CC-0004-0012 US SP KCST Chicnchill 20221007 HUY 1,elephant art,EXACT,472,8,...,0,0,0.000000,0,0,0.00,0.00,8084.0,1.843735e+14,7.461988e+13
629,2022-11-15,2022-11-21,1. NEW,USD,CC-0033-0002 US SP KW PHRASE Chinchill 2022111...,CC-0033-0002 US SP KW PHRASE Chinchill 2022111...,boho wall basket,PHRASE,1578,6,...,0,0,0.000000,0,0,0.00,0.00,9686.0,9.276617e+13,6.669500e+13
630,2022-11-20,2022-12-09,1. NEW,USD,CC-0034-0001 US NEW KW MATK PHRASE Chicnchill ...,CC-0034-0001 US NEW KW MATK PHRASE Chicnchill ...,round woven basket,PHRASE,3689,8,...,0,0,0.000000,0,0,0.00,0.00,1496.0,2.437362e+14,3.118749e+13
631,2022-11-28,2022-12-09,1. NEW,USD,CC-0034-0001 US SP KCST Chinchill 20221128 HUY 4,CC-0034-0001 US SP KCST Chinchill 20221128 HUY 4,boho wall basket decor,EXACT,865,8,...,0,0,0.000000,0,0,0.00,0.00,15302.0,1.264292e+14,2.693971e+14


In [60]:
df_final_merge = get_keyword_Bid(filter_withCamID=df_merge,bulk_df=df_bulk)
df_final_merge

Len Final Merge:      Start Date   End Date Portfolio name Currency  \
0   2022-10-04 2022-11-13        3. DCR2      USD   
1   2022-10-04 2022-11-13        3. DCR2      USD   
2   2022-10-04 2022-11-13        3. DCR2      USD   
3   2022-10-04 2022-11-13        3. DCR2      USD   
4   2022-10-04 2022-11-13        3. DCR2      USD   
..         ...        ...            ...      ...   
822 2022-10-07 2022-10-29        2. DCR1      USD   
823 2022-11-15 2022-11-21         1. NEW      USD   
824 2022-11-20 2022-12-09         1. NEW      USD   
825 2022-11-28 2022-12-09         1. NEW      USD   
826 2022-12-10 2022-12-13         1. NEW      USD   

                                         Campaign Name  \
0    CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...   
1    CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...   
2    CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...   
3    CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...   
4    CC-0031-0002 US SP NEW KW ALL Chicnchill 20211... 

,Start Date,End Date,Portfolio name,Currency,Campaign Name,Ad Group Name,Targeting,Match Type,Impressions,Clicks,...,Campaign Id_y,Ad Group Id_y,Portfolio Id,Keyword Id,Product Targeting Id,Portfolio Name,Keyword Text,Campaign State,Ad Group State,Bid
0,2022-10-04,2022-11-13,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,wall basket decor,EXACT,41544,110,...,2.722893e+14,3.844219e+13,NaN,251660519513810.0,NaN,3. DCR2,wall basket decor,paused,enabled,0.51
1,2022-10-04,2022-11-13,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,wall basket decor,EXACT,41544,110,...,2.722893e+14,3.844219e+13,NaN,33195510636843.0,NaN,3. DCR2,wall basket decor,paused,enabled,1.19
2,2022-10-04,2022-11-13,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,wall basket decor,EXACT,41544,110,...,2.722893e+14,3.844219e+13,NaN,198991724147618.0,NaN,3. DCR2,wall basket decor,paused,enabled,1.18
3,2022-10-04,2022-11-13,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,wall basket decor,EXACT,41544,110,...,2.722893e+14,3.844219e+13,NaN,187847181635987.0,NaN,3. DCR2,wall basket decor,paused,enabled,NaN
4,2022-10-04,2022-11-13,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,wall basket decor,EXACT,41544,110,...,2.722893e+14,3.844219e+13,NaN,11615637024754.0,NaN,3. DCR2,wall basket decor,paused,enabled,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,2022-10-07,2022-10-29,2. DCR1,USD,CC-0004-0012 US SP KCST Chicnchill 20221007 HUY 1,CC-0004-0012 US SP KCST Chicnchill 20221007 HUY 1,elephant art,EXACT,472,8,...,1.843735e+14,7.461988e+13,NaN,220881439284006.0,NaN,2. DCR1,elephant art,paused,enabled,1.50
823,2022-11-15,2022-11-21,1. NEW,USD,CC-0033-0002 US SP KW PHRASE Chinchill 2022111...,CC-0033-0002 US SP KW PHRASE Chinchill 2022111...,boho wall basket,PHRASE,1578,6,...,9.276617e+13,6.669500e+13,NaN,264379633931311.0,NaN,1. NEW,boho wall basket,paused,enabled,1.65
824,2022-11-20,2022-12-09,1. NEW,USD,CC-0034-0001 US NEW KW MATK PHRASE Chicnchill ...,CC-0034-0001 US NEW KW MATK PHRASE Chicnchill ...,round woven basket,PHRASE,3689,8,...,2.437362e+14,3.118749e+13,NaN,224095270633288.0,NaN,1. NEW,round woven basket,paused,enabled,1.41
825,2022-11-28,2022-12-09,1. NEW,USD,CC-0034-0001 US SP KCST Chinchill 20221128 HUY 4,CC-0034-0001 US SP KCST Chinchill 20221128 HUY 4,boho wall basket decor,EXACT,865,8,...,1.264292e+14,2.693971e+14,NaN,130380274452525.0,NaN,1. NEW,boho wall basket decor,paused,enabled,2.10


In [57]:
df_merge.drop_duplicates(inplace=True)
df_merge

,Start Date,End Date,Portfolio name,Currency,Campaign Name,Ad Group Name,Targeting,Match Type,Impressions,Clicks,...,7 Day Total Orders (#),7 Day Total Units (#),7 Day Conversion Rate,7 Day Advertised SKU Units (#),7 Day Other SKU Units (#),7 Day Advertised SKU Sales,7 Day Other SKU Sales,index,Campaign Id,Ad Group Id
0,2022-10-04,2022-11-13,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,wall basket decor,EXACT,41544,110,...,1,2,0.009091,0,2,0.00,47.98,11758.0,2.722893e+14,3.844219e+13
1,2022-10-04,2022-11-11,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,wall hanging baskets,EXACT,7014,21,...,1,1,0.047619,0,1,0.00,34.99,11758.0,2.722893e+14,3.844219e+13
2,2022-10-27,2022-11-06,3. DCR2,USD,CC-0027-0003 US SP ASIN Chicnchill 20221028 TO...,CC-0027-0003 SP US ASIN Chicnchill 20221028 TO...,"asin=""B084VL1XFN""",-,5519,37,...,2,2,0.054054,2,0,59.98,0.00,5505.0,1.423994e+14,2.442783e+14
3,2022-10-05,2022-12-13,3. DCR2,USD,CC-0027-0003 US SP ASIN Chicnchill 20221028 TO...,CC-0027-0003 SP US ASIN Chicnchill 20221028 TO...,"asin=""B09771BDVV""",-,46578,304,...,16,17,0.052632,9,8,264.91,267.42,5505.0,1.423994e+14,2.442783e+14
4,2022-10-05,2022-10-27,3. DCR2,USD,CC-0027-0003 US SP ASIN Chicnchill 20221028 TO...,CC-0027-0003 SP US ASIN Chicnchill 20221028 TO...,"asin=""B092ZMT3KQ""",-,8454,52,...,2,2,0.038462,2,0,64.98,0.00,5505.0,1.423994e+14,2.442783e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,2022-10-07,2022-10-29,2. DCR1,USD,CC-0004-0012 US SP KCST Chicnchill 20221007 HUY 1,CC-0004-0012 US SP KCST Chicnchill 20221007 HUY 1,elephant art,EXACT,472,8,...,0,0,0.000000,0,0,0.00,0.00,8084.0,1.843735e+14,7.461988e+13
629,2022-11-15,2022-11-21,1. NEW,USD,CC-0033-0002 US SP KW PHRASE Chinchill 2022111...,CC-0033-0002 US SP KW PHRASE Chinchill 2022111...,boho wall basket,PHRASE,1578,6,...,0,0,0.000000,0,0,0.00,0.00,9686.0,9.276617e+13,6.669500e+13
630,2022-11-20,2022-12-09,1. NEW,USD,CC-0034-0001 US NEW KW MATK PHRASE Chicnchill ...,CC-0034-0001 US NEW KW MATK PHRASE Chicnchill ...,round woven basket,PHRASE,3689,8,...,0,0,0.000000,0,0,0.00,0.00,1496.0,2.437362e+14,3.118749e+13
631,2022-11-28,2022-12-09,1. NEW,USD,CC-0034-0001 US SP KCST Chinchill 20221128 HUY 4,CC-0034-0001 US SP KCST Chinchill 20221128 HUY 4,boho wall basket decor,EXACT,865,8,...,0,0,0.000000,0,0,0.00,0.00,15302.0,1.264292e+14,2.693971e+14


In [62]:
df_final_merge.to_excel('test_downbid_2.xlsx')

In [21]:
columns = ['Product','Entity','Operation','Campaign Id', 'Ad Group Id',	'Portfolio Id','Ad Id','Keyword Id','Product Targeting Id',	
            'Campaign Name',	'Ad Group Name',	'Start Date',	'End Date',	'Targeting Type',	'State',	'Daily Budget',	
            'SKU',	'ASIN','Ad Group Default Bid',	'Bid',	'Keyword Text',	'Match Type',	'Bidding Strategy',	'Placement',	'Percentage',
            'Product Targeting Expression']
# Product	Entity	Operation	Campaign Id	Ad Group Id	Portfolio Id	Ad Id	Keyword Id	Product Targeting Id	Campaign Name	Ad Group Name	Start Date	End Date	
# Targeting Type	State	Daily Budget	SKU	ASIN	Ad Group Default Bid	Bid	Keyword Text	Match Type	Bidding Strategy	Placement	Percentage	Product Targeting Expression
def create_row_downbid_asin(columns:list,camp_id,ad_group_id,bid):
    row = {}
    for item in columns:
        if item == 'Entity':
            row[item] = 'Product Targeting'
        elif item == 'Product':
            row[item] = 'Sponsored Products'
        elif item =='Operation':
            row[item] = 'Update'
        elif item == 'Campaign Id':
            row[item] = camp_id
        elif item == 'Ad Group Id':
            row[item] = ad_group_id
        elif item == 'Bid':
            row[item] = bid
        elif item == 'Ad Group Default Bid':
            row[item] = str(float(bid)*0.85)
        else:
            row[item] = None
    return row

def create_row_downbid_keyword(columns:list,camp_id,ad_group_id,bid):
    row = {}
    for item in columns:
        if item == 'Entity':
            row[item] = 'Keyword'
        elif item == 'Product':
            row[item] = 'Sponsored Products'
        elif item =='Operation':
            row[item] = 'Update'
        elif item == 'Campaign Id':
            row[item] = camp_id
        elif item == 'Ad Group Id':
            row[item] = ad_group_id
        elif item == 'Bid':
            row[item] = bid
        elif item == 'Ad Group Default Bid':
            row[item] = str(float(bid)*0.85)
        else:
            row[item] = None
    return row

In [22]:
n1 = create_row_downbid_asin(columns=columns,camp_id='1234567',ad_group_id='ABC1234',bid='20')
n2 = create_row_downbid_keyword(columns=columns,camp_id='1234567',ad_group_id='ABC1234',bid='20')

In [34]:
def export_excel_files(filtered_df:pd.DataFrame,columns:list,old_bid):
    result_df = pd.DataFrame(columns=columns)
    for i in range(len(filtered_df)):
        if filtered_df.iloc[i]['Targeting'].startswith('asin'):
            temp_asin_row = create_row_downbid_asin(columns = columns,camp_id=filtered_df.iloc[i]['Campaign Id'],ad_group_id=filtered_df.iloc[i]['Ad Group Id'],bid=old_bid)
            result_df = result_df.append(temp_asin_row,ignore_index=True)
        else:
            temp_kw_row = create_row_downbid_keyword(columns = columns,camp_id=filtered_df.iloc[i]['Campaign Id'],ad_group_id=filtered_df.iloc[i]['Ad Group Id'],bid=old_bid)
            result_df = result_df.append(temp_kw_row,ignore_index=True)         
    return result_df

In [35]:
df_excel = export_excel_files(columns=columns,filtered_df=df_result,old_bid=100)

In [36]:
df_excel

,Product,Entity,Operation,Campaign_Id,Ad_Group_Id,Portfolio_Id,Ad_Id_Read_only,Keyword_Id_Read_only,Product_Targeting_Id_Read_only,Campaign_Name_Informational_only,...,SKU,ASIN_Informational_only,Bid,Ad_Group_Default_Bid,Keyword_Text,Match_Type,Bidding_Strategy,Placement,Percentage,Product_Targeting_Expression
0,Sponsored Products,Keyword,Update,272289318897853,3.844219e+13,None,None,None,None,None,...,None,None,100,85.0,None,None,None,None,None,None
1,Sponsored Products,Keyword,Update,272289318897853,3.844219e+13,None,None,None,None,None,...,None,None,100,85.0,None,None,None,None,None,None
2,Sponsored Products,Product Targeting,Update,142399443597021,2.442783e+14,None,None,None,None,None,...,None,None,100,85.0,None,None,None,None,None,None
3,Sponsored Products,Product Targeting,Update,142399443597021,2.442783e+14,None,None,None,None,None,...,None,None,100,85.0,None,None,None,None,None,None
4,Sponsored Products,Product Targeting,Update,142399443597021,2.442783e+14,None,None,None,None,None,...,None,None,100,85.0,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,Sponsored Products,Keyword,Update,184373499979487,7.461988e+13,None,None,None,None,None,...,None,None,100,85.0,None,None,None,None,None,None
846,Sponsored Products,Keyword,Update,92766168666757,6.669500e+13,None,None,None,None,None,...,None,None,100,85.0,None,None,None,None,None,None
847,Sponsored Products,Keyword,Update,243736237808569,3.118749e+13,None,None,None,None,None,...,None,None,100,85.0,None,None,None,None,None,None
848,Sponsored Products,Keyword,Update,126429228556525,2.693971e+14,None,None,None,None,None,...,None,None,100,85.0,None,None,None,None,None,None


In [37]:
df_excel.to_excel('test_downbid.xlsx')

In [57]:
def read_bulk_report_SD(excel_file=''):
    bulk_df = pd.read_excel(excel_file,index_col=False)
    df_process = bulk_df[['Campaign Id','Ad Group Id','Campaign Name (Informational only)','Ad Group Name (Informational only)','Targeting Expression']]
    df_process.columns = ['Campaign Id','Ad Group Id','Campaign Name','Ad Group Name','Targeting Expression']
    # df_process = bulk_df
    return df_process

In [58]:
df_bulk_SD = read_bulk_report_SD('/Users/vankhaido/BigData/FA/Ads-aws-excel/sample_files/BULK SD Targeting.xlsx')
df_bulk_SD.columns

Index(['Campaign Id', 'Ad Group Id', 'Campaign Name', 'Ad Group Name',
       'Targeting Expression'],
      dtype='object')

In [59]:
def filter_SD_negative(CSP_df: pd.DataFrame,acos,spend):
    filerted_df_1 = CSP_df[CSP_df['Total Advertising Cost of Sales (ACOS) ']>acos]
    filtered_df_3 = CSP_df[CSP_df['Spend']>spend][CSP_df['14 Day Total Sales ']==0]
    list_frames = [filerted_df_1,filtered_df_3]
    filtered_df = pd.concat(list_frames)
    filtered_df.drop_duplicates(inplace=True)
    return filtered_df


In [60]:
df_SD = pd.read_excel('../sample_files/SD Targeting report.xlsx')

/usr/local/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [61]:
df_SD_filter = filter_SD_negative(df_SD,0.6,10)
df_SD_filter

/var/folders/sr/lt1xdrls5_98kvw_5_1t783c0000gn/T/ipykernel_15514/2869564215.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df_3 = CSP_df[CSP_df['Spend']>spend][CSP_df['14 Day Total Sales ']==0]


,Start Date,End Date,Currency,Campaign Name,Portfolio name,Cost Type,Ad Group Name,Targeting,Bid Optimization,Impressions,...,14 Day New-to-brand Sales,14 Day New-to-brand Units (#),Total Advertising Cost of Sales (ACOS) - (Click),Total Return on Advertising Spend (ROAS) - (Click),14 Day Total Orders (#) - (Click),14 Day Total Units (#) - (Click),14 Day Total Sales - (Click),14 Day New-to-brand Orders (#) - (Click),14 Day New-to-brand Sales - (Click),14 Day New-to-brand Units (#) - (Click)
433,2022-11-10,2022-12-05,USD,CC-0031-0002 US SD TARGETING CVR Chicnchill 20...,1. NEW,CPC,CC-0031-0002 US SD TARGETING CVR Chicnchill 20...,views=(similar-product lookback=30),SD_CONVERSIONS,136484.0,...,288.97,3.0,0.89729,1.114466,3.0,3.0,288.97,3.0,288.97,3.0
431,2022-11-10,2022-11-26,USD,CC-0035-0001 US SD TARGETING CVR Chicnchill 20...,1. NEW,CPC,CC-0035-0001 US SD TARGETING CVR Chicnchill 20...,views=(similar-product lookback=30),SD_CONVERSIONS,21786.0,...,0.00,0.0,NaN,0.000000,0.0,0.0,0.00,0.0,0.00,0.0
436,2022-11-10,2022-11-26,USD,CC-0029-0001 US SD TARGETING CVR Chicnchill 20...,1. NEW,CPC,CC-0029-0001 US SD TARGETING CVR Chicnchill 20...,views=(similar-product lookback=30),SD_CONVERSIONS,9861.0,...,0.00,0.0,NaN,0.000000,0.0,0.0,0.00,0.0,0.00,0.0
679,2022-12-01,2022-12-04,USD,CC-0001-0003 US SD ASIN REACH Chicnchill 20221...,2. DCR1,VCPM,CC-0001-0003 US SD ASIN REACH Chicnchill 20221...,similar-product,SD_REACH,21805.0,...,0.00,0.0,NaN,0.000000,0.0,0.0,0.00,0.0,0.00,0.0
681,2022-12-01,2022-12-05,USD,CC-0026-0001 US SD ABRAND REACH Chicnchill 202...,1. NEW,VCPM,CC-0026-0001 US SD ABRAND REACH Chicnchill 202...,similar-product,SD_REACH,26312.0,...,0.00,0.0,NaN,0.000000,0.0,0.0,0.00,0.0,0.00,0.0


In [63]:
df_sp_targeting.head(2)

,Start Date,End Date,Portfolio name,Currency,Campaign Name,Ad Group Name,Targeting,Match Type,Impressions,Clicks,...,Total Advertising Cost of Sales (ACOS),Total Return on Advertising Spend (ROAS),7 Day Total Sales,7 Day Total Orders (#),7 Day Total Units (#),7 Day Conversion Rate,7 Day Advertised SKU Units (#),7 Day Other SKU Units (#),7 Day Advertised SKU Sales,7 Day Other SKU Sales
0,2022-10-05,2022-12-10,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,hanging woven wall basket,EXACT,12019.0,26.0,...,0.155922,6.413447,165.98,2.0,2.0,0.076923,2.0,0.0,165.98,0.00
1,2022-10-04,2022-11-13,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,wall basket decor,EXACT,41544.0,110.0,...,1.533556,0.652079,47.98,1.0,2.0,0.009091,0.0,2.0,0.00,47.98


In [101]:
start_date = df_sp_targeting.iloc[0]['Start Date']

In [69]:
from datetime import datetime

In [102]:
x = start_date.date()
type(x)

datetime.date

In [104]:
td = date.today()

type(td)


datetime.date

In [105]:
x2 = td - x 
x2

datetime.timedelta(days=81)

In [99]:
# if x2 > datetime.timedelta(days=15):
#     print("1")
# else:
#     print("0")
type(x2)
type(x2.days)

int

In [147]:
if '14 Day Total Sales ' in df_SD.columns.to_list():
    print(True)
else:
    print(False)

True


In [153]:
# Filter Tang bid

# def filter_row_date(row):
#     if (date.today() - row['Start Date'].date()).days > 7:
#         return True
#     else:
#         return False

# def filter_upbid(CSP_df: pd.DataFrame,dates_diff,impressions):
#     def filter_row_date(row):
#         if (date.today() - row['Start Date'].date()).days > dates_diff:
#             return True
#         else:
#             return False
#     m = CSP_df.apply(filter_row_date,axis=1)
#     filtered_df_1 = CSP_df[m]
#     filtered_df_2 = CSP_df[CSP_df['Impressions'] <impressions][CSP_df['14 Day Total Sales ']==0]
#     list_frames = [filtered_df_1,filtered_df_2]
#     filtered_df = pd.concat(list_frames)
#     filtered_df.drop_duplicates(inplace=True)
#     return filtered_df
def filter_upbid(CSP_df: pd.DataFrame,dates_diff,impressions):
    def filter_row_date(row):
        if (date.today() - row['Start Date'].date()).days > dates_diff:
            return True
        else:
            return False
    m = CSP_df.apply(filter_row_date,axis=1)
    filtered_df_1 = CSP_df[m]
    if '14 Day Total Sales ' in CSP_df.columns.to_list():
        filtered_df_2 = CSP_df[CSP_df['Impressions'] <impressions][CSP_df['14 Day Total Sales ']==0]
        list_frames = [filtered_df_1,filtered_df_2]
    elif '7 Day Total Sales ' in CSP_df.columns.to_list():
        filtered_df_2 = CSP_df[CSP_df['Impressions'] <impressions][CSP_df['7 Day Total Sales ']==0]
        list_frames = [filtered_df_1,filtered_df_2]
    else:
        list_frames = [filtered_df_1]
    
    filtered_df = pd.concat(list_frames)
    filtered_df.drop_duplicates(inplace=True)
    return filtered_df

In [152]:
df_upbid = filter_upbid(df_sp_targeting,dates_diff=48,impressions=100)
df_upbid

/var/folders/sr/lt1xdrls5_98kvw_5_1t783c0000gn/T/ipykernel_15514/3874139005.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df_2 = CSP_df[CSP_df['Impressions'] >impressions][CSP_df['7 Day Total Sales ']==0]


,Start Date,End Date,Portfolio name,Currency,Campaign Name,Ad Group Name,Targeting,Match Type,Impressions,Clicks,...,Total Advertising Cost of Sales (ACOS),Total Return on Advertising Spend (ROAS),7 Day Total Sales,7 Day Total Orders (#),7 Day Total Units (#),7 Day Conversion Rate,7 Day Advertised SKU Units (#),7 Day Other SKU Units (#),7 Day Advertised SKU Sales,7 Day Other SKU Sales
0,2022-10-05,2022-12-10,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,hanging woven wall basket,EXACT,12019.0,26.0,...,0.155922,6.413447,165.98,2.0,2.0,0.076923,2.0,0.0,165.98,0.00
1,2022-10-04,2022-11-13,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,wall basket decor,EXACT,41544.0,110.0,...,1.533556,0.652079,47.98,1.0,2.0,0.009091,0.0,2.0,0.00,47.98
2,2022-10-08,2022-12-05,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,boho wall baskets,EXACT,16170.0,72.0,...,0.207076,4.829144,464.95,5.0,5.0,0.069444,4.0,1.0,359.96,104.99
3,2022-10-04,2022-12-10,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,black wall basket,EXACT,5614.0,13.0,...,0.051812,19.300530,254.96,4.0,4.0,0.307692,4.0,0.0,254.96,0.00
4,2022-10-05,2022-11-20,3. DCR2,USD,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,CC-0031-0002 US SP NEW KW ALL Chicnchill 20211...,large wall basket decor boho,EXACT,18.0,0.0,...,NaN,NaN,0.00,0.0,0.0,NaN,0.0,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2022-12-11,2022-12-13,1. NEW,USD,CC-0032-0002 US SP KW PHRASE Chicnchill 202212...,CC-0032-0002 US SP KW PHRASE Chicnchill 202212...,wall basket decor,PHRASE,341.0,1.0,...,NaN,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.00,0.00
4338,2022-12-13,2022-12-13,1. NEW,USD,CC-0035-0001 US SP KW PHRASE Chicnchill 202212...,CC-0035-0001 US SP KW PHRASE Chicnchill 202212...,wall hanging baskets,PHRASE,189.0,0.0,...,NaN,NaN,0.00,0.0,0.0,NaN,0.0,0.0,0.00,0.00
4340,2022-12-13,2022-12-13,1. NEW,USD,CC-0035-0001 US SP KW PHRASE Chicnchill 202212...,CC-0035-0001 US SP KW PHRASE Chicnchill 202212...,basket wall decor,PHRASE,904.0,7.0,...,NaN,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.00,0.00
4342,2022-12-13,2022-12-13,1. NEW,USD,CC-0035-0001 US SP KW PHRASE Chicnchill 202212...,CC-0035-0001 US SP KW PHRASE Chicnchill 202212...,home wall decor,PHRASE,505.0,6.0,...,NaN,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.00,0.00


In [129]:
df_upbid[0]=False
df_upbid

0      False
1       True
2       True
3       True
4       True
       ...  
678     True
679     True
680     True
681     True
682     True
Length: 683, dtype: bool

In [131]:
df_1 = df_SD[df_upbid]
df_1

,Start Date,End Date,Currency,Campaign Name,Portfolio name,Cost Type,Ad Group Name,Targeting,Bid Optimization,Impressions,...,14 Day New-to-brand Sales,14 Day New-to-brand Units (#),Total Advertising Cost of Sales (ACOS) - (Click),Total Return on Advertising Spend (ROAS) - (Click),14 Day Total Orders (#) - (Click),14 Day Total Units (#) - (Click),14 Day Total Sales - (Click),14 Day New-to-brand Orders (#) - (Click),14 Day New-to-brand Sales - (Click),14 Day New-to-brand Units (#) - (Click)
1,2022-11-06,2022-11-07,USD,ChicnChill-SD-Product-ASIN-WallBaskets,-,VCPM,ChicnChill-SD-Cons-Product-ASIN-WallBaskets,"asin=""B092ZMT3KQ""",SD_REACH,0.0,...,0.00,0.0,NaN,NaN,0.0,0.0,0.00,0.0,0.00,0.0
2,2022-11-06,2022-11-11,USD,ChicnChill-SD-Product-ASIN-WallBaskets,-,VCPM,ChicnChill-SD-Cons-Product-ASIN-WallBaskets,"asin=""B084VL1XFN""",SD_REACH,0.0,...,0.00,0.0,NaN,NaN,0.0,0.0,0.00,0.0,0.00,0.0
3,2022-11-08,2022-11-08,USD,ChicnChill-SD-Product-ASIN-WallBaskets,-,VCPM,ChicnChill-SD-Cons-Product-ASIN-WallBaskets,"asin=""B096WBLPMB""",SD_REACH,0.0,...,0.00,0.0,NaN,NaN,0.0,0.0,0.00,0.0,0.00,0.0
4,2022-11-11,2022-11-11,USD,ChicnChill-SD-Product-ASIN-WallBaskets,-,VCPM,ChicnChill-SD-Cons-Product-ASIN-WallBaskets,"asin=""B09948173R""",SD_REACH,0.0,...,0.00,0.0,NaN,NaN,0.0,0.0,0.00,0.0,0.00,0.0
5,2022-11-07,2022-11-07,USD,ChicnChill-SD-Product-ASIN-WallBaskets,-,VCPM,ChicnChill-SD-Cons-Product-ASIN-WallBaskets,"asin=""B08YZ52X4P""",SD_REACH,0.0,...,0.00,0.0,NaN,NaN,0.0,0.0,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,2022-11-30,2022-11-30,USD,CC-0001-0003 US SD ASIN REACH Chicnchill 20221...,2. DCR1,VCPM,CC-0001-0003 US SD ASIN REACH Chicnchill 20221...,"asin=""B09DFLKKPK""",SD_REACH,7.0,...,0.00,0.0,NaN,0.000000,0.0,0.0,0.00,0.0,0.00,0.0
679,2022-12-01,2022-12-04,USD,CC-0001-0003 US SD ASIN REACH Chicnchill 20221...,2. DCR1,VCPM,CC-0001-0003 US SD ASIN REACH Chicnchill 20221...,similar-product,SD_REACH,21805.0,...,0.00,0.0,NaN,0.000000,0.0,0.0,0.00,0.0,0.00,0.0
680,2022-11-30,2022-12-05,USD,CC-0026-0002 US SD ABRAND REACH Chicnchill 202...,1. NEW,VCPM,CC-0026-0002 US SD ABRAND REACH Chicnchill 202...,"asin=""B08KV9L335""",SD_REACH,213.0,...,9.99,1.0,0.026795,37.320681,1.0,1.0,9.99,1.0,9.99,1.0
681,2022-12-01,2022-12-05,USD,CC-0026-0001 US SD ABRAND REACH Chicnchill 202...,1. NEW,VCPM,CC-0026-0001 US SD ABRAND REACH Chicnchill 202...,similar-product,SD_REACH,26312.0,...,0.00,0.0,NaN,0.000000,0.0,0.0,0.00,0.0,0.00,0.0
